# Misattribution Detection with MLflow Faithfulness Metrics

This notebook demonstrates how to use MLflow faithfulness metrics to detect **misattribution** in LLM outputs. Misattribution occurs when the model incorrectly assigns facts, actions, quotes, or accomplishments to the wrong entity.

## What is Misattribution?

Misattribution is a specific type of hallucination where:
- **Actions are assigned to wrong actors** (e.g., "Einstein invented the telephone")
- **Quotes are attributed to wrong people** (e.g., "Shakespeare said 'I have a dream'")
- **Accomplishments are credited to wrong entities** (e.g., "Microsoft created the iPhone")
- **Properties are associated with wrong subjects** (e.g., "The Nile is in South America")

### Why Misattribution Matters:
| Impact | Example |
|--------|---------|
| **Misinformation** | Crediting wrong inventor spreads false history |
| **Legal Issues** | Wrong company attribution can cause disputes |
| **Credibility Loss** | Users lose trust in AI systems |
| **Harmful Decisions** | Wrong medical/legal attribution can cause harm |

### Faithfulness Score for Misattribution:
| Score Range | Interpretation | Action |
|-------------|----------------|--------|
| >= 0.70 | Correct Attribution | Safe |
| 0.50 - 0.70 | Possible Issues | Review |
| 0.30 - 0.50 | Likely Misattribution | Flag |
| < 0.30 | Clear Misattribution | Block |


## 1. Setup and Installation


In [ ]:
%pip install -q mlflow sentence-transformers pandas numpy scikit-learn


In [ ]:
%restart_python


In [ ]:
import numpy as np
import pandas as pd
import re
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Libraries loaded successfully!")


## 2. Misattribution Detector

We use the same faithfulness metrics as hallucination detection, but focus specifically on entity-fact relationships. The NLI model is particularly effective at detecting when facts are attributed to wrong entities.


In [ ]:
class MisattributionDetector:
    """
    Detects misattribution errors where facts, actions, or quotes are 
    incorrectly attributed to wrong entities.
    """

    def __init__(self, 
                 embedding_model: str = "all-MiniLM-L6-v2",
                 nli_model: str = "cross-encoder/nli-deberta-v3-small"):
        """Initialize with embedding and NLI models."""
        print(f"Loading embedding model: {embedding_model}...")
        self.embedding_model = SentenceTransformer(embedding_model)
        
        print(f"Loading NLI model: {nli_model}...")
        self.nli_model = CrossEncoder(nli_model)
        
        print("✅ Misattribution detector ready!")

    def compute_semantic_faithfulness(self, answer: str, context: str) -> float:
        """Compute faithfulness using semantic similarity."""
        answer_embedding = self.embedding_model.encode([answer])[0]
        context_embedding = self.embedding_model.encode([context])[0]
        similarity = cosine_similarity([answer_embedding], [context_embedding])[0][0]
        return max(0, min(1, (similarity + 1) / 2))

    def compute_nli_faithfulness(self, answer: str, context: str) -> float:
        """Compute faithfulness using NLI entailment."""
        scores = self.nli_model.predict([(context, answer)])[0]
        if isinstance(scores, (int, float)):
            return 1 / (1 + np.exp(-scores))  # sigmoid
        return float(scores[2])  # entailment score

    def compute_token_overlap(self, answer: str, context: str) -> float:
        """Compute simple token overlap faithfulness."""
        stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 
                      'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from',
                      'and', 'but', 'or', 'it', 'its', 'this', 'that', 'who'}
        
        answer_tokens = set(answer.lower().split()) - stop_words
        context_tokens = set(context.lower().split()) - stop_words
        
        if not answer_tokens:
            return 1.0
        
        overlap = answer_tokens.intersection(context_tokens)
        return len(overlap) / len(answer_tokens)

    def detect_misattribution(self, answer: str, context: str) -> Dict:
        """
        Detect if an answer contains misattribution.
        
        Returns:
            Dictionary with scores and misattribution verdict
        """
        semantic = self.compute_semantic_faithfulness(answer, context)
        nli = self.compute_nli_faithfulness(answer, context)
        overlap = self.compute_token_overlap(answer, context)
        
        # Combined score (NLI weighted heavily for attribution detection)
        combined = 0.55 * nli + 0.30 * semantic + 0.15 * overlap
        
        # Determine misattribution category
        if combined >= 0.70:
            category = "✅ Correct Attribution"
            is_misattribution = False
            risk = "Low"
        elif combined >= 0.50:
            category = "⚠️ Possible Misattribution"
            is_misattribution = True
            risk = "Medium"
        elif combined >= 0.30:
            category = "❌ Likely Misattribution"
            is_misattribution = True
            risk = "High"
        else:
            category = "🚫 Clear Misattribution"
            is_misattribution = True
            risk = "Critical"
        
        return {
            "semantic_score": semantic,
            "nli_score": nli,
            "overlap_score": overlap,
            "combined_score": combined,
            "category": category,
            "is_misattribution": is_misattribution,
            "risk_level": risk
        }

# Initialize the detector
detector = MisattributionDetector()


## 3. Test Examples: Detecting Misattributions

We'll test several categories of misattribution:
1. **Inventor/Discovery Misattribution** - Wrong person credited for inventions
2. **Quote Misattribution** - Quotes assigned to wrong speakers
3. **Company/Product Misattribution** - Products credited to wrong companies
4. **Historical Event Misattribution** - Events attributed to wrong actors
5. **Geographic Misattribution** - Locations associated with wrong places


In [ ]:
# Define test examples with context, correct answer, and misattributed answer
test_examples = [
    {
        "name": "Example 1: Inventor Misattribution",
        "category": "Invention",
        "context": "Thomas Edison invented the practical incandescent light bulb in 1879. He conducted thousands of experiments at his Menlo Park laboratory before achieving success.",
        "correct_answer": "Thomas Edison invented the practical incandescent light bulb in 1879 after thousands of experiments at Menlo Park.",
        "misattributed_answer": "Nikola Tesla invented the practical incandescent light bulb in 1879 after thousands of experiments at Menlo Park."
    },
    {
        "name": "Example 2: Quote Misattribution",
        "category": "Quote",
        "context": "Martin Luther King Jr. delivered his famous 'I Have a Dream' speech during the March on Washington in 1963. The speech became a defining moment of the civil rights movement.",
        "correct_answer": "Martin Luther King Jr. gave the 'I Have a Dream' speech during the 1963 March on Washington.",
        "misattributed_answer": "Abraham Lincoln gave the 'I Have a Dream' speech during the 1963 March on Washington."
    },
    {
        "name": "Example 3: Company/Product Misattribution",
        "category": "Company",
        "context": "Apple Inc. introduced the iPhone in 2007. Steve Jobs unveiled the revolutionary smartphone at the Macworld Conference, describing it as three products in one.",
        "correct_answer": "Apple and Steve Jobs introduced the iPhone at Macworld in 2007, calling it three products in one.",
        "misattributed_answer": "Microsoft and Bill Gates introduced the iPhone at Macworld in 2007, calling it three products in one."
    },
    {
        "name": "Example 4: Scientific Discovery Misattribution",
        "category": "Discovery",
        "context": "Marie Curie discovered polonium and radium. She won two Nobel Prizes: one in Physics (1903) and one in Chemistry (1911), becoming the first person to win Nobel Prizes in two different sciences.",
        "correct_answer": "Marie Curie discovered polonium and radium, winning Nobel Prizes in both Physics and Chemistry.",
        "misattributed_answer": "Albert Einstein discovered polonium and radium, winning Nobel Prizes in both Physics and Chemistry."
    },
    {
        "name": "Example 5: Historical Event Misattribution",
        "category": "History",
        "context": "Neil Armstrong became the first human to walk on the Moon on July 20, 1969. He spoke the famous words 'That's one small step for man, one giant leap for mankind' as he stepped onto the lunar surface.",
        "correct_answer": "Neil Armstrong was the first person to walk on the Moon in 1969, saying 'one small step for man, one giant leap for mankind.'",
        "misattributed_answer": "Buzz Aldrin was the first person to walk on the Moon in 1969, saying 'one small step for man, one giant leap for mankind.'"
    },
    {
        "name": "Example 6: Geographic Misattribution",
        "category": "Geography",
        "context": "The Amazon River, located in South America, is the largest river by discharge volume. It flows through Brazil, Peru, and Colombia, and its basin contains the world's largest rainforest.",
        "correct_answer": "The Amazon River is in South America, flowing through Brazil, Peru, and Colombia with the world's largest rainforest.",
        "misattributed_answer": "The Amazon River is in Africa, flowing through Nigeria, Congo, and Egypt with the world's largest rainforest."
    },
    {
        "name": "Example 7: Author Misattribution",
        "category": "Literature",
        "context": "William Shakespeare wrote Romeo and Juliet around 1594-1596. The tragic love story has become one of the most performed plays in history and influenced countless adaptations.",
        "correct_answer": "William Shakespeare wrote Romeo and Juliet in the 1590s, creating one of history's most performed plays.",
        "misattributed_answer": "Charles Dickens wrote Romeo and Juliet in the 1590s, creating one of history's most performed plays."
    }
]

print(f"📋 Loaded {len(test_examples)} misattribution test examples across {len(set(e['category'] for e in test_examples))} categories")


### 3.1 Running Misattribution Detection


In [ ]:
def print_detection_result(name: str, category: str, context: str, answer: str, result: Dict, answer_type: str):
    """Pretty print the misattribution detection result."""
    print(f"\n{'='*80}")
    print(f"📝 {name}")
    print(f"   Category: {category} | Type: {answer_type}")
    print(f"{'='*80}")
    print(f"\n📄 Context: {context[:100]}...")
    print(f"\n💬 Answer: {answer}")
    print(f"\n📊 SCORES:")
    print(f"   • Semantic Similarity: {result['semantic_score']:.3f}")
    print(f"   • NLI Entailment:      {result['nli_score']:.3f}")
    print(f"   • Token Overlap:       {result['overlap_score']:.3f}")
    print(f"   • Combined Score:      {result['combined_score']:.3f}")
    print(f"\n🎯 VERDICT: {result['category']}")
    print(f"   • Is Misattribution: {'YES ❌' if result['is_misattribution'] else 'NO ✓'}")
    print(f"   • Risk Level: {result['risk_level']}")

# Run detection on all examples
print("🔍 MISATTRIBUTION DETECTION RESULTS")
print("="*80)

all_results = []

for example in test_examples:
    # Test correct attribution
    correct_result = detector.detect_misattribution(
        example["correct_answer"], 
        example["context"]
    )
    print_detection_result(
        example["name"], 
        example["category"],
        example["context"], 
        example["correct_answer"], 
        correct_result, 
        "CORRECT ATTRIBUTION"
    )
    all_results.append({
        "example": example["name"],
        "category": example["category"],
        "type": "Correct",
        "combined_score": correct_result["combined_score"],
        "is_misattribution": correct_result["is_misattribution"],
        "verdict": correct_result["category"]
    })
    
    # Test misattributed answer
    misattributed_result = detector.detect_misattribution(
        example["misattributed_answer"], 
        example["context"]
    )
    print_detection_result(
        example["name"], 
        example["category"],
        example["context"], 
        example["misattributed_answer"], 
        misattributed_result, 
        "MISATTRIBUTED ANSWER"
    )
    all_results.append({
        "example": example["name"],
        "category": example["category"],
        "type": "Misattributed",
        "combined_score": misattributed_result["combined_score"],
        "is_misattribution": misattributed_result["is_misattribution"],
        "verdict": misattributed_result["category"]
    })


### 3.2 Summary Results Table


In [ ]:
# Create summary DataFrame
results_df = pd.DataFrame(all_results)

print("\n" + "="*80)
print("📊 SUMMARY: MISATTRIBUTION DETECTION RESULTS")
print("="*80)

# Format the dataframe for display
display_df = results_df.copy()
display_df["combined_score"] = display_df["combined_score"].apply(lambda x: f"{x:.3f}")
display_df["is_misattribution"] = display_df["is_misattribution"].apply(lambda x: "❌ YES" if x else "✓ NO")
display_df.columns = ["Example", "Category", "Answer Type", "Score", "Misattribution?", "Verdict"]

print("\n")
print(display_df.to_string(index=False))

# Calculate accuracy
correct_identified = sum(1 for r in all_results if r["type"] == "Correct" and not r["is_misattribution"])
misattributed_detected = sum(1 for r in all_results if r["type"] == "Misattributed" and r["is_misattribution"])
total = len(test_examples)

print(f"\n{'='*80}")
print("📈 DETECTION ACCURACY")
print("="*80)
print(f"   • Correct attributions identified:     {correct_identified}/{total} ({correct_identified/total*100:.0f}%)")
print(f"   • Misattributions detected:            {misattributed_detected}/{total} ({misattributed_detected/total*100:.0f}%)")
print(f"   • Overall accuracy:                    {(correct_identified+misattributed_detected)/(total*2)*100:.0f}%")


### 3.3 Analysis by Misattribution Category


In [ ]:
# Analyze by category
print("\n" + "="*80)
print("📊 ANALYSIS BY MISATTRIBUTION CATEGORY")
print("="*80)

categories = results_df["category"].unique()

for cat in categories:
    cat_results = results_df[results_df["category"] == cat]
    correct_score = cat_results[cat_results["type"] == "Correct"]["combined_score"].values[0]
    misattr_score = cat_results[cat_results["type"] == "Misattributed"]["combined_score"].values[0]
    score_gap = correct_score - misattr_score
    
    print(f"\n📌 {cat}:")
    print(f"   Correct Attribution Score:     {correct_score:.3f}")
    print(f"   Misattributed Score:           {misattr_score:.3f}")
    print(f"   Detection Gap:                 {score_gap:.3f} {'✓' if score_gap > 0.15 else '⚠️'}")
    
# Overall statistics
print(f"\n{'='*80}")
print("📊 OVERALL STATISTICS")
print("="*80)

correct_scores = results_df[results_df["type"] == "Correct"]["combined_score"]
misattr_scores = results_df[results_df["type"] == "Misattributed"]["combined_score"]

print(f"\n📗 CORRECT ATTRIBUTION SCORES:")
print(f"   Mean:  {correct_scores.mean():.3f}")
print(f"   Min:   {correct_scores.min():.3f}")
print(f"   Max:   {correct_scores.max():.3f}")

print(f"\n📕 MISATTRIBUTED SCORES:")
print(f"   Mean:  {misattr_scores.mean():.3f}")
print(f"   Min:   {misattr_scores.min():.3f}")
print(f"   Max:   {misattr_scores.max():.3f}")

print(f"\n📉 SEPARATION:")
avg_gap = correct_scores.mean() - misattr_scores.mean()
print(f"   Average Gap: {avg_gap:.3f}")
print(f"   Overlap Risk: {'Low ✓' if avg_gap > 0.25 else 'Medium ⚠️' if avg_gap > 0.15 else 'High ❌'}")


## 4. Interactive Testing: Try Your Own Examples


In [ ]:
def test_misattribution(context: str, answer: str):
    """
    Test if an answer contains misattribution given a context.
    
    Usage:
        test_misattribution(
            context="Albert Einstein developed the theory of relativity.",
            answer="Einstein developed the theory of relativity."
        )
    """
    result = detector.detect_misattribution(answer, context)
    
    print("="*60)
    print("🔍 MISATTRIBUTION TEST")
    print("="*60)
    print(f"\n📄 Context:\n{context}")
    print(f"\n💬 Answer:\n{answer}")
    print(f"\n📊 SCORES:")
    print(f"   • Semantic: {result['semantic_score']:.3f}")
    print(f"   • NLI:      {result['nli_score']:.3f}")
    print(f"   • Overlap:  {result['overlap_score']:.3f}")
    print(f"   • Combined: {result['combined_score']:.3f}")
    print(f"\n🎯 RESULT: {result['category']}")
    print(f"   Risk Level: {result['risk_level']}")
    print("="*60)
    
    return result

# Example: Correct attribution
print("TEST 1: Correct Attribution")
test_misattribution(
    context="Isaac Newton formulated the laws of motion and universal gravitation in his work Principia Mathematica published in 1687.",
    answer="Isaac Newton developed the laws of motion and universal gravitation in Principia Mathematica (1687)."
)


In [ ]:
# Example: Misattribution (wrong person)
print("TEST 2: Misattribution - Wrong Person")
test_misattribution(
    context="Isaac Newton formulated the laws of motion and universal gravitation in his work Principia Mathematica published in 1687.",
    answer="Galileo Galilei developed the laws of motion and universal gravitation in Principia Mathematica (1687)."
)


In [ ]:
# Example: Misattribution (wrong company)
print("TEST 3: Misattribution - Wrong Company")
test_misattribution(
    context="Google developed the Android operating system. Android was first released in 2008 and has become the world's most widely used smartphone platform.",
    answer="Samsung developed the Android operating system, first released in 2008 as the world's most widely used smartphone platform."
)


## 5. Understanding Misattribution Detection Challenges

Misattribution is particularly tricky because the answer often contains factually correct information - just attributed to the wrong entity. This creates high semantic overlap but should still be flagged.


In [ ]:
# Demonstrate the challenge: High semantic overlap but wrong attribution
print("="*70)
print("🔬 CHALLENGE CASE: HIGH OVERLAP, WRONG ENTITY")
print("="*70)

challenge_context = """
Jeff Bezos founded Amazon in 1994 as an online bookstore in his garage 
in Bellevue, Washington. The company later expanded to sell everything 
and became one of the world's most valuable companies.
"""

correct_answer = "Jeff Bezos founded Amazon in 1994 as an online bookstore, which grew to become one of the most valuable companies."
misattributed_answer = "Elon Musk founded Amazon in 1994 as an online bookstore, which grew to become one of the most valuable companies."

print("\n📄 Context:", challenge_context.strip())
print("\n--- Testing Correct Attribution ---")
correct_result = detector.detect_misattribution(correct_answer, challenge_context)
print(f"💬 Answer: {correct_answer}")
print(f"📊 Scores: Semantic={correct_result['semantic_score']:.3f}, NLI={correct_result['nli_score']:.3f}, Overlap={correct_result['overlap_score']:.3f}")
print(f"🎯 Combined: {correct_result['combined_score']:.3f} → {correct_result['category']}")

print("\n--- Testing Misattributed Answer ---")
misattr_result = detector.detect_misattribution(misattributed_answer, challenge_context)
print(f"💬 Answer: {misattributed_answer}")
print(f"📊 Scores: Semantic={misattr_result['semantic_score']:.3f}, NLI={misattr_result['nli_score']:.3f}, Overlap={misattr_result['overlap_score']:.3f}")
print(f"🎯 Combined: {misattr_result['combined_score']:.3f} → {misattr_result['category']}")

print("\n📌 KEY INSIGHT:")
print(f"   Both answers share similar structure and high word overlap.")
print(f"   However, the NLI model detects the entity mismatch:")
print(f"   • NLI score difference: {correct_result['nli_score'] - misattr_result['nli_score']:.3f}")
print(f"   • This is why NLI is weighted heavily (55%) for misattribution detection.")


## 6. Score Comparison Visualization


In [ ]:
# Visual comparison of scores
correct_scores = results_df[results_df["type"] == "Correct"]["combined_score"]
misattr_scores = results_df[results_df["type"] == "Misattributed"]["combined_score"]

print("="*60)
print("📊 SCORE COMPARISON: CORRECT vs MISATTRIBUTED")
print("="*60)

print("\n📗 CORRECT ATTRIBUTIONS:")
print(f"   Average Score: {np.mean(correct_scores):.3f}")
print(f"   Min Score:     {np.min(correct_scores):.3f}")
print(f"   Max Score:     {np.max(correct_scores):.3f}")

print("\n📕 MISATTRIBUTED ANSWERS:")
print(f"   Average Score: {np.mean(misattr_scores):.3f}")
print(f"   Min Score:     {np.min(misattr_scores):.3f}")
print(f"   Max Score:     {np.max(misattr_scores):.3f}")

print("\n📉 SEPARATION METRICS:")
score_diff = np.mean(correct_scores) - np.mean(misattr_scores)
print(f"   Score Gap: {score_diff:.3f}")

# Visual bar representation
print("\n📊 VISUAL COMPARISON:")
print(f"   Correct:      {'█' * int(np.mean(correct_scores) * 20):<20} {np.mean(correct_scores):.2f}")
print(f"   Misattributed:{'█' * int(np.mean(misattr_scores) * 20):<20} {np.mean(misattr_scores):.2f}")
print(f"   Threshold:    {'─' * 10}│{'─' * 9}  0.50 (misattribution cutoff)")

# Per-category visualization
print("\n📊 BY CATEGORY:")
for cat in categories:
    cat_data = results_df[results_df["category"] == cat]
    c_score = cat_data[cat_data["type"] == "Correct"]["combined_score"].values[0]
    m_score = cat_data[cat_data["type"] == "Misattributed"]["combined_score"].values[0]
    print(f"   {cat:12} Correct: {'█' * int(c_score * 15):<15} {c_score:.2f}  |  Misattr: {'█' * int(m_score * 15):<15} {m_score:.2f}")


## 7. Key Takeaways


In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                    KEY TAKEAWAYS: MISATTRIBUTION DETECTION                   ║
╚══════════════════════════════════════════════════════════════════════════════╝

✅ WHAT WE DEMONSTRATED:

1. MISATTRIBUTION TYPES DETECTED:
   • Inventor/Discovery attribution errors
   • Quote misattribution (wrong speaker)
   • Company/Product attribution errors
   • Historical event misattribution
   • Geographic location errors
   • Author/Creator misattribution

2. WHY NLI IS CRITICAL:
   • Semantic similarity alone fails for misattribution
   • Both correct and wrong attributions have high word overlap
   • NLI detects entailment violations when entities mismatch
   • Weighted 55% in combined score for this reason

3. DETECTION CHALLENGES:
   • High semantic similarity between correct and wrong answers
   • Token overlap is nearly identical
   • Only entity names differ - requires understanding of context
   • NLI models excel at catching these contradictions

4. RECOMMENDED THRESHOLDS FOR MISATTRIBUTION:
   • >= 0.70: Safe - correct attribution
   • 0.50-0.70: Review - possible issues
   • 0.30-0.50: Flag - likely misattribution
   • < 0.30: Block - clear misattribution

═══════════════════════════════════════════════════════════════════════════════

🔗 NEXT STEPS:
   • Integrate with MLflow for production monitoring
   • Set up entity-specific validation rules
   • Combine with Named Entity Recognition (NER) for entity extraction
   • Build domain-specific attribution validators

═══════════════════════════════════════════════════════════════════════════════

⚠️ LIMITATIONS:
   • Requires good NLI model understanding of entities
   • May struggle with lesser-known entities
   • Performance depends on context quality and completeness
   • Consider ensemble approaches for high-stakes applications

═══════════════════════════════════════════════════════════════════════════════
""")
